In [1]:
import requests
import pandas as pd

from dotenv import load_dotenv
import os
load_dotenv()

gameName = "BakutoPhinn"
tagLine = "NA1"
api_key = os.environ.get("riot_api")

In [2]:
def get_puuid_ign(gameName, tagLine, api_key):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}"
    response = requests.get(link)
    return response.json()['puuid']

def get_puuid_sID(sID, api_key):
    link = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{sID}?api_key={api_key}"
    response = requests.get(link)
    return response.json()['puuid']

def get_name_and_tag(puuid, api_key):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
    response = requests.get(link)
    return [response.json()['gameName'], response.json()['tagLine']]
# must use strings challengerleagues, grandmasterleagues, or masterleagues for league input
def get_league(league, api_key):
    link = f"https://na1.api.riotgames.com/lol/league/v4/{league}/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
    return requests.get(link).json()

In [3]:
def get_ladder():
    chall_response = get_league("challengerleagues", api_key)
    chall_df = pd.DataFrame(chall_response['entries'])
    chall_df = chall_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    gm_response = get_league("grandmasterleagues", api_key)
    gm_df = pd.DataFrame(gm_response['entries'])
    gm_df = gm_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    masters_response = get_league("masterleagues", api_key)
    masters_df = pd.DataFrame(masters_response['entries'])
    masters_df = masters_df.sort_values("leaguePoints", ascending=False).reset_index(drop=True)
    ladder = pd.concat([chall_df, gm_df, masters_df])
    ladder = ladder.drop(columns='rank')
    ladder = ladder.reset_index()
    ladder = ladder.rename(columns={"index":"Rank"})
    ladder["Rank"] = ladder["Rank"]+1
    return ladder

In [9]:
def get_match_history(puuid, api_key, start=0, count=20, type=""):
    if type != "":
        link = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start={start}&count={count}&api_key={api_key}"
    else:
        link = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={api_key}"
    print(link)
    return requests.get(link).json()

def get_match_data(matchID, api_key):
    link = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchID}?api_key={api_key}"
    return requests.get(link).json()

def get_usernames(matchID, api_key):
    data = get_match_data(matchID, api_key)
    usernames_puuid = data['metadata']['participants']
    usernames = []
    for puuid in usernames_puuid:
        usernames.append(get_name_and_tag(puuid, api_key)[0])
    return usernames

def get_match_KDA(matchID, playerID, api_key):
    data = get_match_data(matchID, api_key)
    participant_puuids = data['metadata']['participants']
    player_idx = 0
    for i in range(10):
        if (participant_puuids[i]==playerID):
            player_idx = i
    player_info = data['info']['participants'][player_idx]
    if (player_info['deaths']==0):
        return player_info['kills']+player_info['assists']
    return (player_info['kills']+player_info['assists'])/player_info['deaths']

def get_player_KDA(gameName, tagLine, api_key):
    puuid = get_puuid_ign(gameName, tagLine, api_key)
    matches = get_match_history(puuid, api_key)
    total = 0
    for i in range(20):
        total += get_match_KDA(matches[i], puuid, api_key)
    return total/20

    

In [11]:
get_player_KDA("jwong02", "NA1", api_key)

https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/t_RtfJyUg_GIC-i0TqnSkWgPOimmvOHgZZsaiNUdNOKy9wiJufU2uWyGhVZpUYdus8d3aMYveJk7MA/ids?start=0&count=20&api_key=RGAPI-2490ba66-d5e5-49c2-a07a-81a55ec32192


2.6457539682539677